In [8]:
from scipy.spatial import cKDTree
from scipy.sparse import identity, issparse

import numpy as np
import pandas as pd
import scanpy as sc

from liana.method.sp._spatial_pipe import spatial_neighbors
from liana.method.sp._misty import _get_distance_weights

from scipy.sparse import csr_matrix

In [14]:
from anndata import AnnData

In [5]:
adata = sc.read_h5ad('liana/tests/data/synthetic.h5ad')

In [183]:
## generate matrix
weights = csr_matrix(np.ones((10, 10)))
mat = csr_matrix(np.ones((10, 5)))
labels = np.random.choice(['A', 'B'], size=10)

In [184]:
mat.todense()

matrix([[1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1.]])

In [193]:
labels

array(['A', 'A', 'B', 'A', 'B', 'B', 'A', 'A', 'B', 'B'], dtype='<U1')

In [186]:
adata = AnnData(mat, obs={'labels': labels}, obsp={'spatial_connectivities': weights.copy()})

In [187]:
(adata.obs['labels']=='A').sum()

5

In [188]:
adata.obsp['spatial_connectivities'].sum()

100.0

In [189]:
adata.obsp['spatial_connectivities'][:,0].sum()

10.0

In [190]:
weights[:,0].sum()

10.0

In [194]:
weights.todense()

matrix([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]])

In [195]:
adata.obsp['spatial_connectivities'][:,adata.obs['labels']!='A']=0

In [196]:
weights.data[np.isin(weights.indices, np.where(adata.obs['labels']!='A')[0])]=0

In [197]:
weights[:,0].sum()

10.0

In [198]:
adata.obsp['spatial_connectivities'][:,0].sum()

10.0

In [199]:
adata.obsp['spatial_connectivities'].todense()

matrix([[1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.],
        [1., 1., 0., 1., 0., 0., 1., 1., 0., 0.]])

In [200]:
(weights @ adata.X).todense()

matrix([[5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.],
        [5., 5., 5., 5., 5.]])

In [161]:
adata.shape

(10, 5)

In [164]:
adata.X

<10x5 sparse matrix of type '<class 'numpy.float64'>'
	with 50 stored elements in Compressed Sparse Row format>